In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import tensorflow as tf
import timeit


class MNISTDD(object):
    def __init__(self, subset='train', batch_size=64, labeled_percent=0.2, shuffle=True):
        X_train, y_train = np.reshape(np.load("train_X.npy"), (55000,64,64,1)), np.load("train_Y.npy")
        X_valid, y_valid = np.reshape(np.load("valid_X.npy"), (5000,64,64,1)), np.load("valid_Y.npy")
#         X_test, y_test = np.reshape(np.load("test_X.npy"), (10000,64,64,1)), np.load("test_Y.npy")
        X_test, y_test = X_train[0:10000], y_train[0:10000]
        
        
        bboxes_train = np.load("train_bboxes.npy")
        bboxes_valid = np.load("valid_bboxes.npy")
        bboxes_test = np.load("test_bboxes.npy")
        
        if subset == 'train':
            images = X_train
            labels = y_train
            bboxes = bboxes_train
            np.random.seed(100)
#             is_labeled = np.zeros((55000, 1))
            labeled_images = np.random.permutation(np.arange(55000))[:int(labeled_percent * 55000)]
#             is_labeled[labeled_images] = 1
            np.random.seed()
  
        elif subset == 'valid':
            images = X_valid
            labels = y_valid
            bboxes = bboxes_valid
#             is_labeled = np.ones((5000, 1))

        elif subset == 'test':
            images = X_test
            labels = y_test
            bboxes = bboxes_test
       
        else:
            raise NotImplementedError
        self._images = images
        self.images = self._images
        self._labels = labels
        self.labels = self._labels
        self._bboxes = bboxes
        self.bboxes = self._bboxes
#         self._is_labeled = is_labeled
#         self.is_labeled = self._is_labeled
        self.batch_size = batch_size
        self.num_samples = len(self.images)
        self.shuffle = shuffle
        if self.shuffle:
            self.shuffle_samples()
        self.next_batch_pointer = 0

    def shuffle_samples(self):
        image_indices = np.random.permutation(np.arange(self.num_samples))
        self.images = self._images[image_indices]
        self.labels = self._labels[image_indices]
        self.bboxes = self._bboxes[image_indices]
#         self.is_labeled = self._is_labeled[image_indices]

    def get_next_batch(self):
        num_samples_left = self.num_samples - self.next_batch_pointer
        if num_samples_left >= self.batch_size:
            x_batch = self.images[self.next_batch_pointer:self.next_batch_pointer + self.batch_size]
            y_batch = self.labels[self.next_batch_pointer:self.next_batch_pointer + self.batch_size]
            batch_bboxes = self.bboxes[self.next_batch_pointer:self.next_batch_pointer + self.batch_size]
#             is_labeled_batch = self.is_labeled[self.next_batch_pointer:self.next_batch_pointer + self.batch_size]
            self.next_batch_pointer += self.batch_size
        else:
            x_partial_batch_1 = self.images[self.next_batch_pointer:self.num_samples]
            y_partial_batch_1 = self.labels[self.next_batch_pointer:self.num_samples]
            bboxes_partial_batch_1 = self.bboxes[self.next_batch_pointer:self.num_samples]
#             is_labeled_batch_1 = self.is_labeled[self.next_batch_pointer:self.num_samples]
            if self.shuffle:
                self.shuffle_samples()
            x_partial_batch_2 = self.images[0:self.batch_size - num_samples_left]
            y_partial_batch_2 = self.labels[0:self.batch_size - num_samples_left]
            bboxes_partial_batch_2 = self.bboxes[0:self.batch_size - num_samples_left]
#             is_labeled_batch_2 = self.is_labeled[0:self.batch_size - num_samples_left]
            x_batch = np.vstack((x_partial_batch_1, x_partial_batch_2))
            y_batch = np.vstack((y_partial_batch_1, y_partial_batch_2))
            batch_bboxes = np.vstack((bboxes_partial_batch_1,bboxes_partial_batch_2))
#             is_labeled_batch = np.vstack((is_labeled_batch_1, is_labeled_batch_2))
            self.next_batch_pointer = self.batch_size - num_samples_left
        return x_batch, y_batch, batch_bboxes

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.layers import flatten
from sklearn.utils import shuffle
from skimage.draw import polygon

"""Ops part"""


# Reference: https://github.com/openai/improved-gan/blob/master/imagenet/ops.py
import tensorflow as tf
import numpy as np


# TODO: Optionally create utility functions for convolution, fully connected layers here
def create_weights(shape):
  initializer = tf.contrib.layers.xavier_initializer() 
  return tf.Variable(initializer(shape))

def create_biases(size):
  return tf.Variable(tf.zeros(size))

def conv2d(x,W):
#   weight_decay = 1e-4
#   regularizer = tf.contrib.layers.12_regularizer(weight_decay)
  return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

def max_pool(x):
  return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1], padding='VALID')

def conv_layer(input,shape):
  W = create_weights(shape)
  b = create_biases(shape[3])
  return tf.nn.elu(conv2d(input,W) + b)

def fc_layer(input,size):
  in_size = int(input.get_shape()[1])
  W = create_weights([in_size,size])
  b = create_biases([size])
  return tf.add(tf.matmul(input,W), b)



In [ ]:
def classification_net(input_tensor, is_training, dropout_kept_prob):
  is_training = tf.convert_to_tensor(is_training, tf.bool)
  initializer = tf.contrib.layers.xavier_initializer()
  
  conv1_filter = 51
  conv1_1 = conv_layer(input_tensor, (3,3,1,conv1_filter))
#   conv1_2 = conv_layer(conv1_1, (3,3,conv1_filter,conv1_filter))
  conv1_pool = max_pool(conv1_1)
  
  conv2_filter = 51
  conv2_1 = conv_layer(conv1_pool, (3,3, conv1_filter,conv2_filter))
#   conv2_2 = conv_layer(conv2_1, (3,3, conv2_filter,conv2_filter))
  conv2_1_pool = max_pool(conv2_1)
  
  conv2_3 = conv_layer(conv2_1_pool, (3,3, conv2_filter,conv2_filter))
  conv2_4 = conv_layer(conv2_3, (3,3,conv2_filter,conv2_filter))
  conv2_pool = max_pool(conv2_4)
  
  conv3_filter = 51
  conv3_1 = conv_layer(conv2_pool, (3,3,conv2_filter, conv3_filter))
  conv3_1_pool = max_pool(conv3_1)
  
  conv3_2 = conv_layer(conv3_1_pool, (3,3,conv3_filter, conv3_filter))
  conv3_2_pool = max_pool(conv3_2)
  
  conv3_3 = conv_layer(conv3_2_pool, (3,3,conv3_filter,conv3_filter))
  conv3_pool = max_pool(conv3_3)
  
  fc0 = flatten(conv3_pool)
  
  fc0_dropout = tf.nn.dropout(fc0, keep_prob=0.5)
  fc_filter = 51
  
  # Conv
  fc4_W = tf.Variable(initializer(shape=(fc_filter,20)), name='fc4_W')
  fc4_b = tf.Variable(tf.zeros(20), name='fc2_b')
  logits = tf.matmul(fc0,fc4_W) + fc4_b
  
  # Pool
  bfc4_W = tf.Variable(initializer(shape=(fc_filter,8)), name='fc4_W')
  bfc4_b = tf.Variable(tf.zeros(8), name='fc2_b')
  blogits = tf.matmul(fc0,bfc4_W) + bfc4_b
  
  return logits, blogits



def train():
  tf.reset_default_graph()

  EPOCHS = 150
  BATCH_SIZE = 64
  NUM_ITERS = int(55000/BATCH_SIZE * EPOCHS)
  print("Number of Iterations: ", NUM_ITERS)
  
  train_set = MNISTDD('train', batch_size=BATCH_SIZE)
  valid_set = MNISTDD('valid',shuffle=False)
  
  X_train, y_train = np.reshape(np.load("train_X.npy"), (55000,64,64,1)), np.load("train_Y.npy")
  bboxes_train = np.load("train_bboxes.npy")

  
  x = tf.placeholder(tf.float32, [None, 64, 64, 1], name='x')
  y = tf.placeholder(tf.int32, [None, 2], name='y')
  bboxes = tf.placeholder(tf.int32,[None,2,4])
  one_hot_y = tf.one_hot(y, depth=10)
  
  rate = 0.001

  logits, blogits = classification_net(x,True,1)
  logits = tf.reshape(logits, [-1,2,10])
  blogits = tf.reshape(blogits, (-1,2,4))
  
  prediction = tf.argmax(logits, axis=2)
  prediction = tf.contrib.framework.sort(prediction)
  bboxes_prediction = blogits

  # Regularization Function: Cross Entropy
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)
  classification_loss = tf.reduce_mean(cross_entropy)
  location_loss = tf.losses.mean_squared_error(bboxes,blogits)
  loss_operation = tf.reduce_mean(cross_entropy) + location_loss
#   loss_operation = tf.reduce_mean(cross_entropy)
  
  
  optimizer = tf.train.AdamOptimizer(learning_rate = rate)
#   grads_and_vars = optimizer.compute_gradients(loss_operation, tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
  training_operation = optimizer.minimize(loss_operation)
  

  correct_prediction = tf.equal(tf.argmax(logits, 2), tf.argmax(one_hot_y, 2))

  accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  saver = tf.train.Saver(max_to_keep=0)
  
    
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    X_train, y_train = shuffle(X_train, y_train)
    for i in range(NUM_ITERS):
      batch_x, batch_y, batch_bboxes = train_set.get_next_batch()
      train_op,acc_op = sess.run([training_operation,accuracy_operation], feed_dict={x: batch_x,
                                                  y: batch_y,
                                                  bboxes: batch_bboxes
                                                  })
      
      if (i + 1) % 1000 == 0 or i == NUM_ITERS - 1:
        cls_pred, bbox_pred,cls_loss, bbox_loss= sess.run([prediction, bboxes_prediction,classification_loss,location_loss], feed_dict={x: valid_set.images,y:valid_set.labels,
                                                                                             bboxes: valid_set.bboxes})
        print("Iteration: ", i)
        print("CLS_LOSS: ", cls_loss)
        print("BBOX_LOSS: ", bbox_loss)
        print("train_acc: ",acc_op)
#       
        evaluation(cls_pred,bbox_pred)
        saver.save(sess,'ckpt/assignment9', global_step=i)
    
    
def test():
  tf.reset_default_graph()
  
  test_set = MNISTDD('test', shuffle=False)
  
  x = tf.placeholder(tf.float32, [None, 64, 64, 1], name='x')
  y_pred = tf.placeholder(tf.int32, [None, 2], name='y')
  bboxes = tf.placeholder(tf.int32,[None,2,4])
  one_hot_y = tf.one_hot(y_pred, depth=10)
  
  logits, blogits = classification_net(x,False,1)
  logits = tf.reshape(logits, [-1,2,10])
  blogits = tf.reshape(blogits, (-1,2,4))
  saver = tf.train.Saver(max_to_keep=0)
  
  
  cls_prediction = tf.argmax(logits, axis=2)
  cls_prediction = tf.contrib.framework.sort(cls_prediction)
  bbox_predictions = blogits
  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #     saver.restore(sess, tf.train.latest_checkpoint('ckpt'))
    saver.restore(sess, './assignment9-128905')
    
    cls_pred, bbox_pred = sess.run([cls_prediction,bbox_predictions], feed_dict={x: test_set.images[0:5000],
                                                                                  y_pred:test_set.labels[0:5000],
                                                                                  bboxes: test_set.bboxes[0:5000]
                                                             })
    
    cls_pred2, bbox_pred2 = sess.run([cls_prediction,bbox_predictions], feed_dict={x: test_set.images[5000:10000],
                                                                                  y_pred:test_set.labels[5000:10000],
                                                                                  bboxes: test_set.bboxes[5000:10000]
                                                             })
    
    cls_pred_final = np.concatenate((cls_pred, cls_pred2))
    bbox_pred_final = np.concatenate((bbox_pred, bbox_pred2))
#     evaluation(cls_pred, bbox_pred,prefix="test")
    evaluation2(cls_pred_final, bbox_pred_final ,prefix="test")  


In [ ]:
def compute_classification_acc(pred, gt):
  # pred and gt are both
  print("pred: ", pred.shape)
  print("gt: ", gt.shape)
  assert pred.shape == gt.shape
  return (pred == gt).astype(int).sum() / gt.size
    
    
def compute_iou(b_pred,b_gt):
  # b_pred: predicted bounding boxes, shape=(n,2,4)
  # b_gt: ground truth bounding boxes, shape=(n,2,4)
    
  n = np.shape(b_gt)[0]
  L_pred = np.zeros((64,64))
  L_gt = np.zeros((64,64))
  iou = 0.0
  for i in range(n):
    for b in range(2):
      rr, cc = polygon([b_pred[i,b,0],b_pred[i,b,0],b_pred[i,b,2],b_pred[i,b,2]],
                   [b_pred[i,b,1],b_pred[i,b,3],b_pred[i,b,3],b_pred[i,b,1]],[64,64])
      L_pred[rr,cc] = 1

      rr, cc = polygon([b_gt[i,b,0],b_gt[i,b,0],b_gt[i,b,2],b_gt[i,b,2]],
                      [b_gt[i,b,1],b_gt[i,b,3],b_gt[i,b,3],b_gt[i,b,1]],[64,64])
      L_gt[rr,cc] = 1

      iou += (1.0/(2*n))*(np.sum((L_pred+L_gt)==2)/np.sum((L_pred+L_gt)>=1))

      L_pred[:,:] = 0
      L_gt[:,:] = 0
    
  return iou


def evaluation(pred_class, pred_bboxes, prefix="valid"):
  # pred_class: Your predicted labels for the 2 digits, shape [N, 2]
  # pred_bboxes: Your predicted bboxes for 2 digits, shape [N, 2, 4]
  gt_class = np.load(prefix + "_Y.npy")
  gt_bboxes = np.load(prefix + "_bboxes.npy")
  acc = compute_classification_acc(pred_class, gt_class)
  iou = compute_iou(pred_bboxes, gt_bboxes)
  print(f"Classification Acc: {acc}")
  print(f"BBoxes IOU: {iou}")
  print("===============================")
  
def evaluation2(pred_class, pred_bboxes, prefix="valid"):
  # pred_class: Your predicted labels for the 2 digits, shape [N, 2]
  # pred_bboxes: Your predicted bboxes for 2 digits, shape [N, 2, 4]
  gt_class = np.load("train" + "_Y.npy")
  gt_bboxes = np.load("train" + "_bboxes.npy")
  
  gt_class = gt_class[0:10000]
  gt_bboxes = gt_bboxes[0:10000]
  acc = compute_classification_acc(pred_class, gt_class)
  iou = compute_iou(pred_bboxes, gt_bboxes)
  print(f"Classification Acc: {acc}")
  print(f"BBoxes IOU: {iou}")
  print("===============================")

In [ ]:
if __name__ == '__main__':
  TRAIN = False
  if TRAIN:
    train()
    print("Testing...")
    test()
  else:
    test()
  
  

INFO:tensorflow:Restoring parameters from ./assignment9-128905
pred:  (10000, 2)
gt:  (10000, 2)
Classification Acc: 0.949
BBoxes IOU: 0.8911935609894861
